# Laboratorio 3

El modelo predice el Attack Type (tipo de ataque) basado en las columnas textuales: 'Scenario Description', 'Tools Used', 'Attack Steps ', 'Vulnerability' y 'Tags'. Estas se combinan en una sola característica de texto y se vectorizan usando TF-IDF.

---
#### Paso 1: Importar librerías y cargar el dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Cargar el dataset
df = pd.read_csv('Attack_Dataset.csv')

# Limpiar columna extra si existe
if 'Unnamed: 15' in df.columns:
    df = df.drop('Unnamed: 15', axis=1)

# Combinar columnas textuales en una sola para vectorización
df['combined_text'] = df['Scenario Description'].fillna('') + ' ' + \
                      df['Tools Used'].fillna('') + ' ' + \
                      df['Attack Steps '].fillna('') + ' ' + \
                      df['Vulnerability'].fillna('') + ' ' + \
                      df['Tags'].fillna('')

# Etiquetas (target)
y = df['Attack Type']

# Codificar etiquetas si son strings (Random Forest maneja strings, pero para consistencia usamos LabelEncoder)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

---
### Paso 2: Preprocesamiento y vectorización

In [ ]:
# Vectorización TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')  # Limitamos a 5000 features para eficiencia
X = vectorizer.fit_transform(df['combined_text'])

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

---
### Paso 3: Crear y entrenar el modelo

In [ ]:
# Crear el modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)  # 100 árboles, usa todos los cores

# Entrenar
model.fit(X_train, y_train)

# Evaluar en test
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

---
### Paso 4: Cómo utilizar el modelo para predicciones <br>
Para hacer predicciones con valores propios o aleatorios:
- **Valores propios:** Proporciona un diccionario con las claves: 'Scenario Description', 'Tools Used', 'Attack Steps ', 'Vulnerability', 'Tags'. Cada uno es un string.
- **Valores aleatorios:** Selecciona una fila aleatoria del dataset y predice sobre ella (para simular).

In [ ]:
def predict_attack_type(new_data, is_dict=True):
    """
    Predice el Attack Type dado un diccionario o una fila aleatoria.
    
    - Si is_dict=True: new_data es un dict con claves: 'Scenario Description', 'Tools Used', 
      'Attack Steps ', 'Vulnerability', 'Tags'.
    - Si is_dict=False: Selecciona una fila aleatoria del dataset.
    """
    if not is_dict:
        # Aleatorio: seleccionar fila random
        random_row = df.sample(1)
        combined = random_row['combined_text'].values[0]
        true_label = random_row['Attack Type'].values[0]
        print(f"Usando fila aleatoria (True Attack Type: {true_label})")
    else:
        # Propios: combinar textos
        combined = new_data.get('Scenario Description', '') + ' ' + \
                   new_data.get('Tools Used', '') + ' ' + \
                   new_data.get('Attack Steps ', '') + ' ' + \
                   new_data.get('Vulnerability', '') + ' ' + \
                   new_data.get('Tags', '')
    
    # Vectorizar
    new_vector = vectorizer.transform([combined])
    
    # Predecir
    pred_encoded = model.predict(new_vector)
    pred_label = label_encoder.inverse_transform(pred_encoded)[0]
    
    return pred_label

# Ejemplo con valores propios
custom_data = {
    'Scenario Description': 'A login form fails to validate input',
    'Tools Used': 'Burp Suite, SQLMap',
    'Attack Steps ': 'Enter payload OR 1=1',
    'Vulnerability': 'Unsanitized input',
    'Tags': 'SQLi, Web Security'
}
print("Predicción propia:", predict_attack_type(custom_data))

# Ejemplo aleatorio
print("Predicción aleatoria:", predict_attack_type(None, is_dict=False))